# 第9章 现代循环神经网络

## 9.1 门控循环单元（GRU） 

### 练习 9.1.1

假设我们只想使用时间步$t'$的输入来预测时间步$t > t'$的输出。对于每个时间步，重置门和更新门的最佳值是什么？

**解答：** 

### 练习 9.1.2

调整和分析超参数对运行时间、困惑度和输出顺序的影响。

**解答：** 

### 练习 9.1.3

比较`rnn.RNN`和`rnn.GRU`的不同实现对运行时间、困惑度和输出字符串的影响。

**解答：** 

### 练习 9.1.4

如果仅仅实现门控循环单元的一部分，例如，只有一个重置门或一个更新门会怎样？

**练习：** 

## 9.2 长短期记忆网络（LSTM） 

### 练习 9.2.1

调整和分析超参数对运行时间、困惑度和输出顺序的影响。

**解答：** 

### 练习 9.2.2

如何更改模型以生成适当的单词，而不是字符序列？

**解答：** 

### 练习 9.2.3

在给定隐藏层维度的情况下，比较门控循环单元、长短期记忆网络和常规循环神经网络的计算成本。要特别注意训练和推断成本。

**解答：** 

### 练习 9.2.4  

既然候选记忆元通过使用$\tanh$函数来确保值范围在$(-1,1)$之间，那么为什么隐状态需要再次使用$\tanh$函数来确保输出值范围在$(-1,1)$之间呢？

**解答：** 

### 练习 9.2.5

实现一个能够基于时间序列进行预测而不是基于字符序列进行预测的长短期记忆网络模型。


**解答：** 

## 9.3 深度循环神经网络 

### 练习 9.3.1 

基于我们在8.5节中讨论的单层实现，尝试从零开始实现两层循环神经网络。

**解答：** 

### 练习 9.3.2

在本节训练模型中，比较使用门控循环单元替换长短期记忆网络后模型的精确度和训练速度。

**解答：** 

### 练习 9.3.3

如果增加训练数据，能够将困惑度降到多低？

**解答：** 

### 练习 9.3.4

在为文本建模时，是否可以将不同作者的源数据合并？有何优劣呢？

**解答：** 

## 9.4 双向循环神经网络 

### 练习 9.4.1

如果不同方向使用不同数量的隐藏单位，$\mathbf{H_t}$的形状会发生怎样的变化？

**解答：** 

### 练习 9.4.2

设计一个具有多个隐藏层的双向循环神经网络。

**解答：** 

### 练习 9.4.3

在自然语言中一词多义很常见。例如，“bank”一词在不同的上下文“i went to the bank to deposit cash”和“i went to the bank to sit down”中有不同的含义。如何设计一个神经网络模型，使其在给定上下文序列和单词的情况下，返回该单词在此上下文中的向量表示？哪种类型的神经网络架构更适合处理一词多义？

**解答：** 

## 9.5 机器翻译与数据集 

### 练习 9.5.1

在`load_data_nmt`函数中尝试不同的`num_examples`参数值。这对源语言和目标语言的词表大小有何影响？

**解答：** 

### 练习 9.5.2

某些语言（例如中文和日语）的文本没有单词边界指示符（例如空格）。对于这种情况，单词级词元化仍然是个好主意吗？为什么？

**解答：** 

## 9.6 编码器-解码器架构

### 练习 9.6.1

假设我们使用神经网络来实现“编码器－解码器”架构，那么编码器和解码器必须是同一类型的神经网络吗？

**解答：** 

### 练习 9.6.2

除了机器翻译，还有其它可以适用于”编码器－解码器“架构的应用吗？

**解答：** 

## 9.7 序列到序列的学习（seq2seq） 

### 练习 9.7.1

试着通过调整超参数来改善翻译效果。

**解答：** 

### 练习 9.7.2

重新运行实验并在计算损失时不使用遮蔽，可以观察到什么结果？为什么会有这个结果？

**解答：** 

### 练习 9.7.3

如果编码器和解码器的层数或者隐藏单元数不同，那么如何初始化解码器的隐状态？

**解答：** 

###  练习 9.7.4

在训练中，如果用前一时间步的预测输入到解码器来代替强制教学，对性能有何影响？

**解答：** 

### 练习 9.7.5

用长短期记忆网络替换门控循环单元重新运行实验。

**解答：** 

### 练习 9.7.6

有没有其他方法来设计解码器的输出层？

**解答：** 

## 9.8 束搜索 

### 练习 9.8.1

我们可以把穷举搜索看作一种特殊的束搜索吗？为什么？

**解答：** 

### 练习 9.8.2

在9.7节中的机器翻译问题中应用束搜索。束宽是如何影响预测的速度和结果的？

**解答：** 

### 练习 9.8.3

在8.5节中，我们基于用户提供的前缀，通过使用语言模型来生成文本。这个例子中使用了哪种搜索策略？可以改进吗？

**解答：** 